In [10]:
import numpy as np
import pandas as pd
import random
import copy
import math
from sklearn.cluster import KMeans
from collections import Counter
import time
import matplotlib.pyplot as plt

In [11]:
rf_path = "./rt-ifttt/rt-ifttt.csv"
rf_data = pd.read_csv(rf_path)

In [12]:
rf_data

,Unnamed: 0,OutdoorUV,OutdoorVisibleRay_Lux,OutdoorTemperature_C,OutdoorHumidity_pct,OutdoorPressure_hPa,IndoorUV,IndoorVisibleRay_Lux,IndoorTemperature_C,IndoorHumidity_pct,IndoorPressure_hPa
0,0,4.21,51624.113281,30.68,27.867188,1012.76,0.07,925.531921,27.26,33.154297,1011.81
1,1,4.20,51675.531250,30.63,27.899414,1012.79,0.07,771.276611,27.26,33.232422,1011.79
2,2,4.24,52035.460938,30.76,27.815430,1012.74,0.05,668.439697,27.29,33.145508,1011.78
3,3,4.23,51984.042969,30.75,27.781250,1012.74,0.05,617.021301,27.30,33.145508,1011.78
4,4,4.24,51881.207031,30.87,27.731445,1012.70,0.05,617.021301,27.29,33.123047,1011.72
...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.73,9101.063477,20.71,40.255859,1013.13,0.06,565.602844,27.24,30.381836,1012.20
9996,9996,0.74,9101.063477,20.72,40.235352,1013.11,0.04,617.021301,27.24,30.392578,1012.15
9997,9997,0.74,9049.645508,20.73,40.368164,1013.15,0.03,565.602844,27.24,30.381836,1012.18
9998,9998,0.74,9101.063477,20.73,40.423828,1013.08,0.05,565.602844,27.24,30.415039,1012.18


In [13]:
weather_humidity_path = "./Historical Hourly Weather Data 2012-2017/humidity.csv"
weather_pressure_path = "./Historical Hourly Weather Data 2012-2017/pressure.csv"
weather_temperature_path = "./Historical Hourly Weather Data 2012-2017/temperature.csv"
weather_wind_speed_path = "./Historical Hourly Weather Data 2012-2017/wind_speed.csv"
weather_humidity_data = pd.read_csv(weather_humidity_path)
weather_pressure_data = pd.read_csv(weather_pressure_path)
weather_temperature_data = pd.read_csv(weather_temperature_path)
weather_wind_speed_data = pd.read_csv(weather_wind_speed_path)
weather_data = [weather_humidity_data, weather_pressure_data, weather_temperature_data, weather_wind_speed_data]
for dataset in weather_data:
    for col_name in dataset.columns:
        # 处理空值
        if dataset[col_name].isnull().any():
            # 用平均值填充
            dataset.fillna({col_name: int(dataset[col_name].mean())}, inplace=True)
            
for idx in range(len(weather_data)):
    weather_data[idx] = weather_data[idx].drop(weather_data[idx].columns[0], axis=1)
weather_data_list = ["_humidity", "_pressure", "_temperature", "_wind_speed"]
for idx_dataset in range(len(weather_data)):   
    for idx in range(len(weather_data[idx_dataset].columns)):
        weather_data[idx_dataset].columns.values[idx] += weather_data_list[idx_dataset]
weather_data_aggregated = pd.concat([weather_data[0]], axis=1)
weather_data_aggregated.columns.values[0] = "Timestamp"
for idx in range(1, len(weather_data)):
    weather_data_aggregated = pd.concat([weather_data_aggregated, weather_data[idx].head(10000)], axis=1)
weather_data_aggregated['Timestamp'] = range(1, len(weather_data_aggregated) + 1)

In [45]:
bridge_21416_DPM_path = "./2021-04-16/2021-04-16 00-DPM.csv"
bridge_21416_HPT_path = "./2021-04-16/2021-04-16 00-HPT.csv"
bridge_21416_RHS_path = "./2021-04-16/2021-04-16 00-RHS.csv"
bridge_21416_ULT_path = "./2021-04-16/2021-04-16 00-ULT.csv"
bridge_21416_VIB_path = "./2021-04-16/2021-04-16 00-VIB.csv"
bridge_21416_VIC_path = "./2021-04-16/2021-04-16 00-VIC.csv"
bridge_21416_DPM_data = pd.read_csv(bridge_21416_DPM_path)
bridge_21416_HPT_data = pd.read_csv(bridge_21416_HPT_path)
bridge_21416_RHS_data = pd.read_csv(bridge_21416_RHS_path)
bridge_21416_ULT_data = pd.read_csv(bridge_21416_ULT_path)
bridge_21416_VIB_data = pd.read_csv(bridge_21416_VIB_path)
bridge_21416_VIC_data = pd.read_csv(bridge_21416_VIC_path)
bridge_data = [bridge_21416_DPM_data, bridge_21416_HPT_data, bridge_21416_RHS_data, bridge_21416_ULT_data, bridge_21416_VIB_data, bridge_21416_VIC_data]
# 找出行数最少的 DataFrame 的行数
min_rows = min(df.shape[0] for df in bridge_data)
# 截断所有 DataFrame 的行数为最小行数
bridge_data = [df.iloc[:min_rows, :] for df in bridge_data]

In [48]:
bridge_data

[       00:00:00.000   103.761   101.761  Unnamed: 3
 0      00:00:00.100   103.763   101.764         NaN
 1      00:00:00.200   103.760   101.763         NaN
 2      00:00:00.300   103.765   101.765         NaN
 3      00:00:00.400   103.764   101.763         NaN
 4      00:00:00.500   103.763   101.762         NaN
 ...             ...       ...       ...         ...
 35984  00:59:58.500   103.667   101.680         NaN
 35985  00:59:58.600   103.669   101.678         NaN
 35986  00:59:58.700   103.668   101.679         NaN
 35987  00:59:58.800   103.667   101.678         NaN
 35988  00:59:58.900   103.672   101.673         NaN
 
 [35989 rows x 4 columns],
        00:00:00.000  4854.628  4799.306  3366.273  3323.542  2026.681  \
 0      00:00:00.100  4854.414  4799.193  3365.402  3321.864  2026.677   
 1      00:00:00.200  4855.120  4799.808  3365.459  3322.957  2026.425   
 2      00:00:00.300  4855.650  4799.917  3365.670  3323.058  2026.338   
 3      00:00:00.400  4856.657  4800.15

In [35]:
for dataset in bridge_data:
    for col_name in dataset.columns:
        # 处理空值
        if dataset[col_name].isnull().any():
            # 用平均值填充
            dataset.fillna({col_name: int(dataset[col_name].mean())}, inplace=True)

ValueError: cannot convert float NaN to integer

In [15]:

for idx in range(len(bridge_data)):
    bridge_data[idx] = bridge_data[idx].drop(bridge_data[idx].columns[0], axis=1)

bridge_data_aggregated = pd.concat([bridge_data[0]], axis=1)
bridge_data_aggregated.columns.values[0] = "Timestamp"
for idx in range(1, len(bridge_data)):
    bridge_data_aggregated = pd.concat([bridge_data_aggregated, bridge_data[idx].head(10000)], axis=1)
bridge_data_aggregated['Timestamp'] = range(1, len(bridge_data_aggregated) + 1)